In [2]:
import quantumpropagator as qp
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets

import os
import sys
import glob
import pandas as pd
import numpy as np

%matplotlib ipympl

plt.rcParams.update({'font.size': 6})

In [5]:
root = '/home/alessio/w-August-Run/'
pump = 'b-UV-0.22_0000'
probe = 'y-probe-UV_0000'

fol_pump = os.path.join(root, pump)
fol_probe = os.path.join(root, probe)
output_norm_pump = os.path.join(fol_pump, 'output')
output_dipo_pump = os.path.join(fol_pump, 'Output_Dipole')
output_norm_probe = os.path.join(fol_probe, 'output')
output_dipo_probe = os.path.join(fol_probe, 'Output_Dipole')


In [69]:
def read_dipole_file(dipole_file_path):
    nstates = 8
    permanents = []
    transitions = []

    for lab1 in ['x','y','z']:
        for lab2 in range(nstates):
            permanents.append('perm_{}_{}'.format(lab1,lab2))
            for lab3 in range(lab2+1,nstates):
                transitions.append('trans_{}_{}_{}'.format(lab1,lab2,lab3))
                
    all_labels = ['fs_2','dipx','dipy','dipz'] +  permanents + transitions
    df_dipo_pump = pd.read_csv(dipole_file_path, delim_whitespace=True, names=all_labels)
    return(df_dipo_pump)

df_norm2_pump = pd.read_csv(output_norm_pump, delim_whitespace=True, index_col=0, names=['counter', 'steps', 'fs','Norm deviation','Kinetic','Potential','Total','Total Deviation','Xpulse','Ypulse','Zpulse','AbZino'])
df_norm2_probe = pd.read_csv(output_norm_probe, delim_whitespace=True, index_col=0, names=['counter', 'steps', 'fs','Norm deviation','Kinetic','Potential','Total','Total Deviation','Xpulse','Ypulse','Zpulse','AbZino'])

df_dipo_pump = read_dipole_file(output_dipo_pump)
df_dipo_probe = read_dipole_file(output_dipo_probe)

In [81]:
def copy_paste_output(pump_df,probe_df, out_name):
    '''
    This function takes a pump and a probe run and returns a new dataframe with pump_probe pasted together
    '''
    fs_cut = probe_df['fs'].iloc[0]
    # the index on the pump sheet is where the first fs value of the probe is.
    index_cut = pump_df.index[pump_df['fs'] == fs_cut][0]
    pump_cut = pump_df.iloc[:index_cut]
    final_paper = pd.concat((pump_cut,probe_df))
    final_paper.to_csv(out_name)
    return(final_paper)

def copy_paste_dipos(pump_df_d,probe_df_d, out_name):
    '''
    This function takes a pump and a probe run and returns a new dataframe with pump_probe pasted together
    '''
    fs_cut = probe_df_d['fs_2'].iloc[0]
    # the index on the pump sheet is where the first fs value of the probe is.
    index_cut = pump_df_d.index[pump_df_d['fs_2'] == fs_cut][0]
    pump_cut = pump_df_d.iloc[:index_cut]
    final_paper = pd.concat((pump_cut,probe_df_d))
    final_paper.to_csv(out_name)
    return(final_paper)

    
ppname = os.path.join('/home/alessio/w-August-Run/','{}_{}.csv'.format(pump,probe))
ppname_dip = os.path.join('/home/alessio/w-August-Run/','{}_{}_dipole.csv'.format(pump,probe))

df1 = copy_paste_output(df_norm2_pump, df_norm2_probe, ppname)
dip1 = copy_paste_dipos(df_dipo_pump, df_dipo_probe, ppname_dip)

In [82]:
# cut the shortest, please
(df1.shape, dip1.shape)
num1, _ = df1.shape
num2, _ = dip1.shape
cut_here = np.amin([num1,num2])
df = df1[:cut_here]
dip = dip1[:cut_here]

In [119]:
def pythonfft(signal_time,signal,pad_length):
    aaa = np.fft.fft(np.pad(signal,(0,pad_length)))
    time_au = qp.fromFsToAu(signal_time)
    dt = time_au[1]- time_au[0]
    bb = np.fft.fftfreq(time_au.size + pad_length)

    time_au_max = np.amax(time_au)
    bbb = qp.fromHartoEv(bb*2*np.pi/dt)
    indexes = np.where(bb>=0)
    return(bbb[indexes],aaa[indexes])

In [120]:

x = df['fs']

pulsex = df['Xpulse']
pulsey = df['Ypulse']
pulsez = df['Zpulse']

together = np.stack((pulsex,pulsey,pulsez), axis=1)
y = np.sum(together,axis=1)

fig, ax0 = plt.subplots(1,1,figsize = (10,6))
ax0.plot(x,y)
fig.tight_layout()

/home/alessio/config/quantumpropagator/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [123]:
pad_length = 100000

freq, signal_fft = pythonfft(np.array(x),np.array(y),pad_length)
fig, ax0 = plt.subplots(1,1,figsize = (10,6))
ax0.plot(freq, abs(signal_fft))
ax0.set_xlim(0,10)
fig.tight_layout()

/home/alessio/config/quantumpropagator/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [128]:
xx = dip['fs_2']
yy = dip['dipx']
freq_dip, signal_fft_dip = pythonfft(np.array(xx),np.array(yy),pad_length)

fig, ax0 = plt.subplots(1,1,figsize = (10,6))
ax0.plot(freq_dip, abs(signal_fft_dip))
ax0.set_xlim(0,10)
fig.tight_layout()

/home/alessio/config/quantumpropagator/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
S_W = -2 * np.imag(np.conj(signal_fft) * signal_fft_dip)

In [130]:
fig, ax0 = plt.subplots(1,1,figsize = (10,6))
ax0.plot(freq_dip, S_W)
ax0.set_xlim(0,10)
fig.tight_layout()

/home/alessio/config/quantumpropagator/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …